# 0. Imports

In [228]:
import pandas as pd
import numpy as np
import re

import sys 
sys.path.append("..")
from src.data_cleaning_support import error_band, convert_to_hours

import ast

# 1. Cleaning of travel tables

# 1.1 Flights

In [229]:
flight_itinieraries1 = pd.read_csv("../data/flights/flight_itinieraries1.csv", index_col=0)
flight_itinieraries2 = pd.read_csv("../data/flights/flight_itinieraries2.csv", index_col=0)

In [230]:
flight_itinieraries1.info()
flight_itinieraries1.head()

<class 'pandas.core.frame.DataFrame'>
Index: 174 entries, 0 to 173
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   duration                    174 non-null    int64  
 1   price                       174 non-null    int64  
 2   price_currency              174 non-null    object 
 3   stops                       174 non-null    int64  
 4   departure                   174 non-null    object 
 5   arrival                     174 non-null    object 
 6   company                     174 non-null    object 
 7   self_transfer               174 non-null    bool   
 8   fare_isChangeAllowed        174 non-null    bool   
 9   fare_isPartiallyChangeable  174 non-null    bool   
 10  fare_isCancellationAllowed  174 non-null    bool   
 11  fare_isPartiallyRefundable  174 non-null    bool   
 12  score                       174 non-null    float64
 13  origin_airport              174 non-null

,duration,price,price_currency,stops,departure,arrival,company,self_transfer,fare_isChangeAllowed,fare_isPartiallyChangeable,fare_isCancellationAllowed,fare_isPartiallyRefundable,score,origin_airport,destination_airport
0,130,225,€,0,2024-11-08 13:20:00,2024-11-08 15:30:00,transavia,False,False,False,False,False,0.999000,Madrid,Paris Orly
1,130,255,€,0,2024-11-08 09:00:00,2024-11-08 11:10:00,transavia,False,False,False,False,False,0.841351,Madrid,Paris Orly
2,115,278,€,0,2024-11-08 16:30:00,2024-11-08 18:25:00,Air Europa,False,False,False,False,False,0.660020,Madrid,Paris Orly
3,115,365,€,0,2024-11-08 08:00:00,2024-11-08 09:55:00,Air Europa,False,False,False,False,False,0.576296,Madrid,Paris Orly
4,115,322,€,0,2024-11-08 14:25:00,2024-11-08 16:20:00,Air Europa,False,False,False,False,False,0.547622,Madrid,Paris Orly


In [231]:
flight_itinieraries2.info()
flight_itinieraries2.head()

<class 'pandas.core.frame.DataFrame'>
Index: 143 entries, 0 to 142
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   duration                    143 non-null    int64  
 1   price                       143 non-null    int64  
 2   price_currency              143 non-null    object 
 3   stops                       143 non-null    int64  
 4   departure                   143 non-null    object 
 5   arrival                     143 non-null    object 
 6   company                     143 non-null    object 
 7   self_transfer               143 non-null    bool   
 8   fare_isChangeAllowed        143 non-null    bool   
 9   fare_isPartiallyChangeable  143 non-null    bool   
 10  fare_isCancellationAllowed  143 non-null    bool   
 11  fare_isPartiallyRefundable  143 non-null    bool   
 12  score                       143 non-null    float64
 13  origin_airport              143 non-null

,duration,price,price_currency,stops,departure,arrival,company,self_transfer,fare_isChangeAllowed,fare_isPartiallyChangeable,fare_isCancellationAllowed,fare_isPartiallyRefundable,score,origin_airport,destination_airport
0,185,167,€,0,2024-11-13 17:10:00,2024-11-13 20:15:00,Ryanair,False,False,False,False,False,0.999000,Madrid,Berlin Brandenburg
1,185,277,€,0,2024-11-13 07:50:00,2024-11-13 10:55:00,Iberia Express,False,False,False,False,False,0.642656,Madrid,Berlin Brandenburg
2,280,211,€,1,2024-11-13 13:00:00,2024-11-13 17:40:00,KLM,False,False,False,False,False,0.601672,Madrid,Berlin Brandenburg
3,185,258,€,0,2024-11-13 20:05:00,2024-11-13 23:10:00,Iberia Express,False,False,False,False,False,0.521972,Madrid,Berlin Brandenburg
4,185,265,€,0,2024-11-13 20:05:00,2024-11-13 23:10:00,Iberia,False,False,False,False,False,0.513911,Madrid,Berlin Brandenburg


Taking a lot at both the info method and the head of the flights dataframes for both cities, the data types and values seem correct. So no cleaning is needed here.

## 1.2 Accommodations

In [232]:
accommodations_city1 = pd.read_csv("../data/accommodations/acommodations_city1.csv",index_col=0)
accommodations_city2 = pd.read_csv("../data/accommodations/acommodations_city2.csv",index_col=0)

In [233]:
accommodations_city1.info()
accommodations_city1.head()

<class 'pandas.core.frame.DataFrame'>
Index: 419 entries, 0 to 418
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   name                     419 non-null    object 
 1   url                      419 non-null    object 
 2   price_currency           419 non-null    object 
 3   total_price_amount       419 non-null    int64  
 4   distance_city_center_km  419 non-null    float64
 5   score                    419 non-null    float64
 6   n_comments               419 non-null    int64  
 7   close_to_metro           419 non-null    object 
 8   sustainability_cert      419 non-null    object 
 9   room_type                419 non-null    object 
 10  double_bed               419 non-null    object 
 11  single_bed               419 non-null    object 
 12  free_cancellation        419 non-null    object 
 13  breakfast_included       419 non-null    object 
 14  pay_at_hotel             419 no

,name,url,price_currency,total_price_amount,distance_city_center_km,score,n_comments,close_to_metro,sustainability_cert,room_type,double_bed,single_bed,free_cancellation,breakfast_included,pay_at_hotel,location_score,free_taxi
0,Five Boutique Hotel Paris Quartier Latin,https://www.booking.com/hotel/fr/the-five.es.h...,€,1175,2.1,8.3,169,Yes,No,Habitación Doble Superior,Yes,No,Yes,Yes,No,NaN,No
1,Sonder Le Frochot,https://www.booking.com/hotel/fr/opera-frochot...,€,1221,2.9,7.6,787,Yes,No,Habitación con cama grande,Yes,No,Yes,Yes,No,NaN,No
2,Appart'City Collection Paris Gare de Lyon,https://www.booking.com/hotel/fr/appart-city-c...,€,896,2.9,9.0,1237,Yes,No,Estudio Doble,Yes,No,Yes,Yes,Yes,NaN,No
3,Le Patio Bastille,https://www.booking.com/hotel/fr/le-patio-st-a...,€,958,3.0,8.3,3037,Yes,No,Habitación Clásica,Yes,Yes,Yes,Yes,No,NaN,No
4,Austin's Saint Lazare Hotel,https://www.booking.com/hotel/fr/austin-s-sain...,€,959,2.9,8.1,3281,Yes,No,Habitación Doble,Yes,No,Yes,Yes,No,NaN,No


In [234]:
accommodations_city2.info()
accommodations_city2.head()

<class 'pandas.core.frame.DataFrame'>
Index: 114 entries, 0 to 113
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   name                     114 non-null    object 
 1   url                      114 non-null    object 
 2   price_currency           114 non-null    object 
 3   total_price_amount       114 non-null    int64  
 4   distance_city_center_km  114 non-null    float64
 5   score                    114 non-null    float64
 6   n_comments               114 non-null    int64  
 7   close_to_metro           114 non-null    object 
 8   sustainability_cert      114 non-null    object 
 9   room_type                114 non-null    object 
 10  double_bed               114 non-null    object 
 11  single_bed               114 non-null    object 
 12  free_cancellation        114 non-null    object 
 13  breakfast_included       114 non-null    object 
 14  pay_at_hotel             114 no

,name,url,price_currency,total_price_amount,distance_city_center_km,score,n_comments,close_to_metro,sustainability_cert,room_type,double_bed,single_bed,free_cancellation,breakfast_included,pay_at_hotel,location_score,free_taxi
0,"InterContinental Berlin, an IHG Hotel",https://www.booking.com/hotel/de/interconti-be...,€,644,2.4,8.8,3828,Yes,No,Habitación Clásica,Yes,Yes,No,No,No,NaN,No
1,"Holiday Inn Express - Berlin - Alexanderplatz,...",https://www.booking.com/hotel/de/holiday-inn-e...,€,687,2.4,8.4,10416,Yes,No,Habitación Estándar,Yes,Yes,Yes,Yes,Yes,NaN,No
2,Hotel Gat Point Charlie,https://www.booking.com/hotel/de/gat-point-cha...,€,676,1.1,8.2,7993,Yes,Yes,Habitación Doble Grande,Yes,Yes,Yes,Yes,Yes,9.4,No
3,"Hotel Berlin, Berlin, a member of Radisson Ind...",https://www.booking.com/hotel/de/bbberlin.es.h...,€,610,2.1,8.0,17744,Yes,Yes,Habitación Estándar,No,Yes,No,No,No,NaN,No
4,Berlin Marriott Hotel,https://www.booking.com/hotel/de/marriott-am-p...,€,1015,0.6,8.6,2826,Yes,Yes,Habitación Deluxe - 1 cama extragrande,Yes,No,Yes,Yes,Yes,9.5,No


Judging from both the info method and the head of the accommodation dataframes for both cities, the data types and values seem correct. So no cleaning is needed here.

## 1.3 Activities

In [235]:
civitatis_activities_city1 = pd.read_csv("../data/activities/civitatis_activities_city1.csv",index_col=0)
civitatis_activities_city2 = pd.read_csv("../data/activities/civitatis_activities_city2.csv",index_col=0)

In [236]:
civitatis_activities_city1.info()
civitatis_activities_city1.head()

<class 'pandas.core.frame.DataFrame'>
Index: 94 entries, 0 to 93
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   activity_name    93 non-null     object 
 1   description      94 non-null     object 
 2   url              93 non-null     object 
 3   image            94 non-null     object 
 4   image2           83 non-null     object 
 5   available_days   94 non-null     object 
 6   available_times  94 non-null     object 
 7   duration         94 non-null     object 
 8   latitude         93 non-null     float64
 9   longitude        93 non-null     float64
 10  address          80 non-null     object 
 11  price            93 non-null     float64
 12  currency         93 non-null     object 
 13  category         78 non-null     object 
dtypes: float64(3), object(11)
memory usage: 11.0+ KB


,activity_name,description,url,image,image2,available_days,available_times,duration,latitude,longitude,address,price,currency,category
0,Free tour por el Barrio Latino,En este free tour por el Barrio Latino descubr...,www.civitatis.com/es/paris/visita-guiada-barri...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/francia/paris/visita-guiad...,"['10', '11', '09', '12']","[['10:30', '15:30'], ['10:30', '15:30'], ['10:...",2h 15m -2h 30m,48.853209,2.343522,"9, Place Saint-Michel, Quartier de la Monnaie,...",2.50,EUR,Español
1,Tour privado por París,"Montmartre, Notre Dame o los Campos Elíseos so...",www.civitatis.com/es/paris/tour-privado-paris/,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/francia/paris/tour-privado...,"['11', '12', '09', '10']","[['9:00', '10:00', '14:00', '16:00'], ['9:00',...",2 -4h,0.000000,0.000000,NaN,34.00,EUR,Español
2,Entrada a la Sainte-Chapelle y Conciergerie,Con esta entrada combinada a la Conciergerie y...,www.civitatis.com/es/paris/entrada-conciergeri...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/francia/paris/entrada-conc...,"['11', '12', '10', '09']","[[], [], [], []]",Entradas,48.855639,2.345388,"Cour du Mai, Quartier Saint-Germain-l'Auxerroi...",5.15,EUR,NaN
3,"Cena en Madame Brasserie, el restaurante de la...","Si buscáis una velada inolvidable, esta cena e...",www.civitatis.com/es/paris/cena-restaurante-to...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/francia/paris/cena-restaur...,"['12', '10', '11', '09']","[['21:00'], ['21:00'], ['21:00'], ['21:00']]",2 horas,48.858537,2.293964,"Tour Eiffel, Quai Jacques Chirac, Quartier du ...",11.77,EUR,Gastronomía y enoturismo
4,Excursión a los Castillos del Loira,Si estáis buscando una preciosa excursión desd...,www.civitatis.com/es/paris/excursion-castillos...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/francia/paris/excursion-ca...,['12'],[['7:00']],12h 30m,48.831234,2.387336,"15, Avenue des Terroirs de France, Quartier de...",37.00,EUR,Español


In [237]:
civitatis_activities_city2.info()
civitatis_activities_city2.head()

<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, 0 to 36
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   activity_name    36 non-null     object 
 1   description      37 non-null     object 
 2   url              36 non-null     object 
 3   image            37 non-null     object 
 4   image2           33 non-null     object 
 5   available_days   37 non-null     object 
 6   available_times  37 non-null     object 
 7   duration         37 non-null     object 
 8   latitude         36 non-null     float64
 9   longitude        36 non-null     float64
 10  address          31 non-null     object 
 11  price            36 non-null     float64
 12  currency         36 non-null     object 
 13  category         30 non-null     object 
dtypes: float64(3), object(11)
memory usage: 4.3+ KB


,activity_name,description,url,image,image2,available_days,available_times,duration,latitude,longitude,address,price,currency,category
0,Free tour de los misterios y leyendas de Berlín,Los secretos de la capital alemana son numeros...,www.civitatis.com/es/berlin/free-tour-misterio...,www.civitatis.com/f/alemania/berlin/free-tour-...,NaN,"['14', '16']","[['10:00'], ['10:00']]",3 horas,52.520625,13.407143,"St. Marienkirche, 8, Karl-Liebknecht-Straße, S...",2.2,EUR,Español
1,Entrada al Icebar Berlín,Si queréis saber cómo se vive dentro de un igl...,www.civitatis.com/es/berlin/entrada-icebar-ber...,www.civitatis.com/f/alemania/berlin/entrada-ic...,NaN,"['16', '14', '15']","[[], [], []]",45 minutos,52.521041,13.404021,"2, Spandauer Straße, Spandauer Vorstadt, Mitte...",5.0,EUR,Entradas
2,Visita guiada por la Isla de los Museos,Acompañadnos en esta visita guiada por la Isla...,www.civitatis.com/es/berlin/visita-guiada-isla...,www.civitatis.com/f/alemania/berlin/visita-gui...,NaN,"['14', '16']","[['10:00'], ['10:00']]",3 horas,52.517258,13.394698,"Unter den Linden, Friedrichswerder, Mitte, Ber...",4.0,EUR,Español
3,Tour en bicicleta por Berlín,En este tour en bicicleta por Berlín practicar...,www.civitatis.com/es/berlin/berlin-bicicleta/,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/alemania/berlin/berlin-bic...,['16'],[['11:00']],3h 30m,52.540055,13.413424,"Kulturbrauerei, 36, Schönhauser Allee, Bremer ...",6.8,EUR,Inglés
4,"Autobús turístico de Berlín, City Sightseeing",El autobús turístico recorre Berlín realizando...,www.civitatis.com/es/berlin/autobus-turistico-...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/alemania/berlin/autobus-tu...,"['15', '16', '14']","[['0:00'], ['0:00'], ['0:00']]",1 -2d,0.000000,0.000000,NaN,9.6,EUR,Español


Here, in contrast to the 2 previous groups of dataframes, the activities dataframes need some re-formatting.

In [238]:
activities_dataframes = [civitatis_activities_city1,civitatis_activities_city2]

for activity_dataframe in activities_dataframes:
    activity_dataframe["image"] = np.where(~activity_dataframe["image2"].isna(), activity_dataframe["image2"],activity_dataframe["image"])

    # make lists literal, as they are str
    activity_dataframe['available_days'] = activity_dataframe['available_days'].apply(ast.literal_eval)
    activity_dataframe['available_times'] = activity_dataframe['available_times'].apply(ast.literal_eval)

    # add a 'd' to account for days when there's a hypen followe by a digit and a 'd'
    activity_dataframe["duration"] = np.where(
    activity_dataframe["duration"].str.contains(r" -\d"), 
    activity_dataframe["duration"].str.replace(r" -(\d)", r"d-\1", regex=True),
    activity_dataframe["duration"]
)

    # split duration into duration min and max
    activity_dataframe[["duration_min","duration_max"]] = activity_dataframe["duration"].str.split("-",expand=True)
    activity_dataframe[["duration_min","duration_max"]]
    activity_dataframe['duration_max'].fillna("0",inplace=True)

    # format irregular durations as hours
    activity_dataframe['hours_min'] = activity_dataframe['duration_min'].apply(convert_to_hours)
    activity_dataframe['hours_max'] = activity_dataframe['duration_max'].apply(convert_to_hours)

    # make min and max equal if no max
    activity_dataframe['hours_max'] = np.where(activity_dataframe['hours_max']==0 | activity_dataframe['hours_max'].isna(), activity_dataframe['hours_min'],activity_dataframe['hours_max'])

    #drop columns and display head
    activity_dataframe.drop(columns=["image2","duration","duration_min","duration_max"],inplace=True)
    print("\n\n\n")
    activity_dataframe.info()

    display(activity_dataframe.head())





<class 'pandas.core.frame.DataFrame'>
Index: 94 entries, 0 to 93
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   activity_name    93 non-null     object 
 1   description      94 non-null     object 
 2   url              93 non-null     object 
 3   image            94 non-null     object 
 4   available_days   94 non-null     object 
 5   available_times  94 non-null     object 
 6   latitude         93 non-null     float64
 7   longitude        93 non-null     float64
 8   address          80 non-null     object 
 9   price            93 non-null     float64
 10  currency         93 non-null     object 
 11  category         78 non-null     object 
 12  hours_min        75 non-null     float64
 13  hours_max        75 non-null     float64
dtypes: float64(5), object(9)
memory usage: 11.0+ KB


C:\Users\Miguel\AppData\Local\Temp\ipykernel_33956\3168278960.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  activity_dataframe['duration_max'].fillna("0",inplace=True)


,activity_name,description,url,image,available_days,available_times,latitude,longitude,address,price,currency,category,hours_min,hours_max
0,Free tour por el Barrio Latino,En este free tour por el Barrio Latino descubr...,www.civitatis.com/es/paris/visita-guiada-barri...,www.civitatis.com/f/francia/paris/visita-guiad...,"[10, 11, 09, 12]","[[10:30, 15:30], [10:30, 15:30], [10:30, 15:30...",48.853209,2.343522,"9, Place Saint-Michel, Quartier de la Monnaie,...",2.50,EUR,Español,362.0,2.5
1,Tour privado por París,"Montmartre, Notre Dame o los Campos Elíseos so...",www.civitatis.com/es/paris/tour-privado-paris/,www.civitatis.com/f/francia/paris/tour-privado...,"[11, 12, 09, 10]","[[9:00, 10:00, 14:00, 16:00], [9:00, 10:00, 14...",0.000000,0.000000,NaN,34.00,EUR,Español,48.0,4.0
2,Entrada a la Sainte-Chapelle y Conciergerie,Con esta entrada combinada a la Conciergerie y...,www.civitatis.com/es/paris/entrada-conciergeri...,www.civitatis.com/f/francia/paris/entrada-conc...,"[11, 12, 10, 09]","[[], [], [], []]",48.855639,2.345388,"Cour du Mai, Quartier Saint-Germain-l'Auxerroi...",5.15,EUR,NaN,NaN,NaN
3,"Cena en Madame Brasserie, el restaurante de la...","Si buscáis una velada inolvidable, esta cena e...",www.civitatis.com/es/paris/cena-restaurante-to...,www.civitatis.com/f/francia/paris/cena-restaur...,"[12, 10, 11, 09]","[[21:00], [21:00], [21:00], [21:00]]",48.858537,2.293964,"Tour Eiffel, Quai Jacques Chirac, Quartier du ...",11.77,EUR,Gastronomía y enoturismo,2.0,2.0
4,Excursión a los Castillos del Loira,Si estáis buscando una preciosa excursión desd...,www.civitatis.com/es/paris/excursion-castillos...,www.civitatis.com/f/francia/paris/excursion-ca...,[12],[[7:00]],48.831234,2.387336,"15, Avenue des Terroirs de France, Quartier de...",37.00,EUR,Español,12.5,12.5






<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, 0 to 36
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   activity_name    36 non-null     object 
 1   description      37 non-null     object 
 2   url              36 non-null     object 
 3   image            37 non-null     object 
 4   available_days   37 non-null     object 
 5   available_times  37 non-null     object 
 6   latitude         36 non-null     float64
 7   longitude        36 non-null     float64
 8   address          31 non-null     object 
 9   price            36 non-null     float64
 10  currency         36 non-null     object 
 11  category         30 non-null     object 
 12  hours_min        29 non-null     float64
 13  hours_max        29 non-null     float64
dtypes: float64(5), object(9)
memory usage: 4.3+ KB


C:\Users\Miguel\AppData\Local\Temp\ipykernel_33956\3168278960.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  activity_dataframe['duration_max'].fillna("0",inplace=True)


,activity_name,description,url,image,available_days,available_times,latitude,longitude,address,price,currency,category,hours_min,hours_max
0,Free tour de los misterios y leyendas de Berlín,Los secretos de la capital alemana son numeros...,www.civitatis.com/es/berlin/free-tour-misterio...,www.civitatis.com/f/alemania/berlin/free-tour-...,"[14, 16]","[[10:00], [10:00]]",52.520625,13.407143,"St. Marienkirche, 8, Karl-Liebknecht-Straße, S...",2.2,EUR,Español,3.00,3.00
1,Entrada al Icebar Berlín,Si queréis saber cómo se vive dentro de un igl...,www.civitatis.com/es/berlin/entrada-icebar-ber...,www.civitatis.com/f/alemania/berlin/entrada-ic...,"[16, 14, 15]","[[], [], []]",52.521041,13.404021,"2, Spandauer Straße, Spandauer Vorstadt, Mitte...",5.0,EUR,Entradas,0.75,0.75
2,Visita guiada por la Isla de los Museos,Acompañadnos en esta visita guiada por la Isla...,www.civitatis.com/es/berlin/visita-guiada-isla...,www.civitatis.com/f/alemania/berlin/visita-gui...,"[14, 16]","[[10:00], [10:00]]",52.517258,13.394698,"Unter den Linden, Friedrichswerder, Mitte, Ber...",4.0,EUR,Español,3.00,3.00
3,Tour en bicicleta por Berlín,En este tour en bicicleta por Berlín practicar...,www.civitatis.com/es/berlin/berlin-bicicleta/,www.civitatis.com/f/alemania/berlin/berlin-bic...,[16],[[11:00]],52.540055,13.413424,"Kulturbrauerei, 36, Schönhauser Allee, Bremer ...",6.8,EUR,Inglés,3.50,3.50
4,"Autobús turístico de Berlín, City Sightseeing",El autobús turístico recorre Berlín realizando...,www.civitatis.com/es/berlin/autobus-turistico-...,www.civitatis.com/f/alemania/berlin/autobus-tu...,"[15, 16, 14]","[[0:00], [0:00], [0:00]]",0.000000,0.000000,NaN,9.6,EUR,Español,24.00,48.00


Data values are now correct, as well as data types and can be used for analysis. Some data like the available days and available times, will have to be exploded in order to be usable for analysis.

# 2. Saving clean data

## 2.1 Flights

In [ ]:
flight_itinieraries1.to_csv("../data/activities/flight_itinieraries1_cleaned.csv")
flight_itinieraries2.to_csv("../data/activities/flight_itinieraries2_cleaned.csv")

## 2.2 Accommodations

In [ ]:
accommodations_city1.to_csv("../data/activities/accommodations_city1_cleaned.csv")
accommodations_city2.to_csv("../data/activities/accommodations_city2_cleaned.csv")

## 2.3 Activities

In [ ]:
civitatis_activities_city1.to_csv("../data/activities/civitatis_activities_city1_cleaned.csv")
civitatis_activities_city2.to_csv("../data/activities/civitatis_activities_city2_cleaned.csv")